#dataset
- Melbourne Housing dataset.

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [36]:
from google.colab import drive
drive.mount('/content/gdrive')

house = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/data/melb_data.csv/melb_data.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#결측치 처리
house = house.dropna(axis=0)

#X, y 데이터 처리
X= house.drop('Price',axis=1)
X = X[['Type','Method','Regionname','Rooms','Distance','Postcode','Bathroom',
      'Landsize','Lattitude','Longtitude','Propertycount']]
y= house.Price

# train,test set 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [38]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
6212,h,S,Eastern Metropolitan,5,13.8,3107.0,2.0,666.0,-37.76920,145.12530,5420.0
10618,t,SA,Western Metropolitan,3,14.8,3023.0,1.0,0.0,-37.70624,144.74042,7719.0
7320,t,S,Southern Metropolitan,2,10.4,3163.0,1.0,0.0,-37.88660,145.03960,2403.0
7086,h,S,Northern Metropolitan,2,14.5,3047.0,1.0,729.0,-37.69090,144.91360,851.0
4690,u,S,Northern Metropolitan,1,9.9,3044.0,1.0,49.0,-37.71520,144.94020,7485.0


In [39]:
X_train.dtypes # Type, Method, Regionname

Type              object
Method            object
Regionname        object
Rooms              int64
Distance         float64
Postcode         float64
Bathroom         float64
Landsize         float64
Lattitude        float64
Longtitude       float64
Propertycount    float64
dtype: object

In [40]:
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print('Categorical variables:',object_cols)

Categorical variables: ['Type', 'Method', 'Regionname']


- 이론적으로 다른 것도 될 수 있음.

score_dataset(): 랜덤포레스트로부터 MAE(mean absolute error)를 구하게 해줌 

In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# function for comparing different approaches
def score_dataset(X_train, X_test, y_train, y_test):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return mean_absolute_error(y_test, preds)

# 1. Drop
- 범주형 데이터 삭제

In [42]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_test = X_test.select_dtypes(exclude=['object'])

print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_test, y_train, y_test))

MAE from Approach 1 (Drop categorical variables):
203418.50797433336


# 2. LabelEncoding

In [43]:
from sklearn.preprocessing import LabelEncoder

# 원데이터 손상을 막기위해 복사
label_X_train = X_train.copy()
label_X_test = X_test.copy()

#label encoder을 각 컬럼에 적용
label_encoder = LabelEncoder()
for col in object_cols:
  label_X_train[col] = label_encoder.fit_transform(X_train[col])
  label_X_test[col] = label_encoder.transform(X_test[col])

print("MAE from Approach 2 (Label Encoding):") 
print(score_dataset(label_X_train, label_X_test, y_train, y_test))

MAE from Approach 2 (Label Encoding):
192952.05123005202


위의 코드 셀에서 각 열에 대해 각 고유값을 임의로 다른 정수에 할당한다. 이것은 맞춤 라벨을 제공하는 것보다 더 간단한 일반적인 접근법이지만,
 
*** 모든 서수 변수(순서가 있는경우)에 대해 더 잘 알고 있는 라벨을 제공한다면 성능의 추가 상승을 기대할 수 있다.***

# 3. One-hot Encoding
- handle_unknown='ignore': 검증 데이터에는 있는데 학습데이터에는 없는 데이터임 -> 무시하자

- sparse = False로 설정하면 OneHotEncoder가 희소 행렬이 아니라 넘파이 배열을 반환함.

더미변수
- 0 또는 1
- 더미변수는 범주형 변수를 연속형 변수로 변환한 것인데, 정확히 따지자면 연속형 변수"스럽게" 만든 것
- 풀랭크 방식 : 더미변수의 값을 원핫인코딩(one-hot-encoding) 방식으로 지정함
  - 범주값이 달라지면 상수항만 달라지고 다른 독립변수의 가중치(영향)는 같은 모형이 되는 것
- 축소랭크 방식: 정한 하나의 범주값을 기준값(reference, baseline)으로 하고 기준값에 대응하는 더미변수의 가중치는 항상 1으로 놓음.
   - 나머지 더미변수의 가중치는 그 상수항에 추가적으로 더해지는 상수항
   - 기준값에 추가적인 특성이 있는 것으로 간주

In [0]:
from patsy import dmatrix

플랭크 방식

In [45]:
dummy_cols_train = pd.DataFrame(X_train[['Type', 'Method', 'Regionname']])
dummy_cols_train.head()

,Type,Method,Regionname
6212,h,S,Eastern Metropolitan
10618,t,SA,Western Metropolitan
7320,t,S,Southern Metropolitan
7086,h,S,Northern Metropolitan
4690,u,S,Northern Metropolitan


In [46]:
# 원래변수의 이름 + [범주값] -> Type[h] ...
dmatrix('Type +0',dummy_cols_train) # +0 또는 -1을 붙여야함.

DesignMatrix with shape (4337, 3)
  Type[h]  Type[t]  Type[u]
        1        0        0
        0        1        0
        0        1        0
        1        0        0
        0        0        1
        1        0        0
        1        0        0
        1        0        0
        1        0        0
        1        0        0
        1        0        0
        1        0        0
        1        0        0
        0        1        0
        1        0        0
        0        1        0
        1        0        0
        1        0        0
        1        0        0
        1        0        0
        0        0        1
        1        0        0
        1        0        0
        0        0        1
        1        0        0
        0        0        1
        1        0        0
        1        0        0
        1        0        0
        1        0        0
  [4307 rows omitted]
  Terms:
    'Type' (columns 0:3)
  (to view full data, use np.asarray(this_

In [47]:
# +0 또는 -1을 설정하지 않으면 축소랭크 방식으로 간주
dmatrix("C(Type,Treatment('h'))",dummy_cols_train)

DesignMatrix with shape (4337, 3)
  Intercept  C(Type, Treatment('h'))[T.t]  C(Type, Treatment('h'))[T.u]
          1                             0                             0
          1                             1                             0
          1                             1                             0
          1                             0                             0
          1                             0                             1
          1                             0                             0
          1                             0                             0
          1                             0                             0
          1                             0                             0
          1                             0                             0
          1                             0                             0
          1                             0                             0
          1                   

____________

In [48]:
object_cols

['Type', 'Method', 'Regionname']

In [0]:
from sklearn.preprocessing import OneHotEncoder

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train  = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_test  = pd.DataFrame(OH_encoder.fit_transform(X_test[object_cols]))

In [0]:
# index가 제거됐기 때문에, 다시 맞춰줘야함.
OH_cols_train.index = X_train.index
OH_cols_test.index = X_test.index


In [0]:
# 범주형 변수를 제거해 주자
num_X_train = X_train.drop(object_cols, axis=1)
num_X_test = X_test.drop(object_cols, axis=1)

#One-hot encoded 컬럼을 추가해 주자
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)

In [61]:
X_train.columns

Index(['Type', 'Method', 'Regionname', 'Rooms', 'Distance', 'Postcode',
       'Bathroom', 'Landsize', 'Lattitude', 'Longtitude', 'Propertycount'],
      dtype='object')

In [63]:
pd.get_dummies(X_train[['Type','Regionname']]) #★

,Type_h,Type_t,Type_u,Regionname_Eastern Metropolitan,Regionname_Eastern Victoria,Regionname_Northern Metropolitan,Regionname_Northern Victoria,Regionname_South-Eastern Metropolitan,Regionname_Southern Metropolitan,Regionname_Western Metropolitan,Regionname_Western Victoria
6212,1,0,0,1,0,0,0,0,0,0,0
10618,0,1,0,0,0,0,0,0,0,1,0
7320,0,1,0,0,0,0,0,0,1,0,0
7086,1,0,0,0,0,1,0,0,0,0,0
4690,0,0,1,0,0,1,0,0,0,0,0
2550,1,0,0,0,0,1,0,0,0,0,0
2252,1,0,0,0,0,0,0,0,1,0,0
4073,1,0,0,0,0,0,0,0,0,1,0
448,1,0,0,0,0,0,0,0,0,1,0
7343,1,0,0,0,0,0,0,0,1,0,0


In [52]:
OH_X_train.head()

,Rooms,Distance,Postcode,Bathroom,Landsize,Lattitude,Longtitude,Propertycount,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
6212,5,13.8,3107.0,2.0,666.0,-37.76920,145.12530,5420.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10618,3,14.8,3023.0,1.0,0.0,-37.70624,144.74042,7719.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7320,2,10.4,3163.0,1.0,0.0,-37.88660,145.03960,2403.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7086,2,14.5,3047.0,1.0,729.0,-37.69090,144.91360,851.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4690,1,9.9,3044.0,1.0,49.0,-37.71520,144.94020,7485.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [68]:
for i in pd.unique(X_train.Type):
  X_train[i] = 0
  X_train.loc[X_train.Type==i, i] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [69]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bathroom,Landsize,Lattitude,Longtitude,Propertycount,h,t,u,1
6212,1,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,0,0,1
10618,1,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1,0,1
7320,1,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1,0,1
7086,1,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,0,0,1
4690,1,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1,1,1


In [53]:
print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_test, y_train, y_test))

MAE from Approach 3 (One-Hot Encoding):
191607.67188273263
